<a href="https://colab.research.google.com/github/turbulent0/leetcode_problems/blob/master/Copy_of_notes_on_actor_critic_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actor Critic Method Notebook

*- Apoorv Nandan*

*Intended for people who are familiar with machine learning and neural networks in general.*

## TLDR Intro to RL
So for those of you that are not at all familiar with reinforcement learning, here's a quick gist:

- In RL, we don't have datasets as such. We instead have an environment, a simulation e.g. a game like flappy bird or dota.
- Then, we train an agent (which is just one or more neural networks), to achieve some sort of an objective within the environment. (like winning the game, or scoring high)



## Agent and Environment

In actor critic method, we have two neural networks inside our agent - an actor and a critic. They can optionaly share parameters as well, as they both operate on the same input - the observations that you get from the environment. These observations are also known as the state of the environment. The environment and the observations are often set up in a way where you can represent all the observations with a tensor, which can be fed into our agent.

Let's consider the hello-world parallel of RL environments - cartpole.

Cartpole environment has two things:
* Cart: A movable cart that can move left or right along a frictionless track.
* Pole: A rigid pole attached to the cart by a hinge, allowing it to swing freely in the vertical plane.

![cartpole image](https://gymnasium.farama.org/_images/cart_pole.gif)

The observations are just 4 numbers which we combine to create an input tensor of shape `(4,)`.

The 4 numbers represent:
* Cart Position (x): The horizontal position of the cart on the track.
* Cart Velocity (x_dot): The velocity of the cart.
* Pole Angle (θ): The angle of the pole with respect to the vertical.
* Pole Angular Velocity (θ_dot): The angular velocity of the pole.

The actor network will take this as the input, and predict probabilities for each of the possible actions in this env, so that the agent can decide what to do based on the observations.

There are just two possible actions here:
* 0: Apply a force to the left.
* 1: Apply a force to the right.

The critic network will take the same input, and predict a single number which we will call the `critic value`. This represents an estimate of the total rewards the agent should be able to get from this point onwards, looking at the current observations from the environment.

So now we can make a loop to make the agent play around inside the environment.

The loop looks like this:
```
               ┌───────┐
   ┌──────────▶│ Agent │────────────┐
   │           └───────┘            │
   │        ┌─────────────┐         │
   └────────│ Environment │◀────────┘
            └─────────────┘
```
The agent chooses an action to perform (based on the probabilities predicted by the actor network). The environment processes that action, and gives back a new set of observations for the agent to act upon.

Let's code this;


In [ ]:
import gym # contains predefined envs that can process the agent's action,
# and return the rewards and the observations.
import numpy as np
np.random.seed(42)
env = gym.make("CartPole-v0")  # create the environment
env.seed(42)

# actor and critic will be simple feed forward networks, and they will share the first layer.
num_inputs = 4
num_actions = 2
num_hidden = 128

def init_xavier_weights(shape):
    return np.random.randn(*shape) * np.sqrt(2 / shape[0])

def init_agent_params():
    w_common = init_xavier_weights((num_inputs, num_hidden))
    w_actor = init_xavier_weights((num_hidden, num_actions))
    w_critic = init_xavier_weights((num_hidden, 1))
    return {
        'w_common': w_common,
        'w_actor': w_actor,
        'w_critic': w_critic
    }

agent_params = init_agent_params()

max_steps_per_episode = 10000
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

while True:
    state = env.reset()
    episode_reward = 0
    for timestep in range(1, max_steps_per_episode):
        # forward pass of agent to get action probs and critic value
        h = np.dot(state, agent_params['w_common'])
        h = np.maximum(h, 0)
        logits = np.dot(h, agent_params['w_actor'])
        action_probs = np.exp(logits) / np.sum(np.exp(logits))

        critic_value = np.dot(h, agent_params['w_critic'])
        critic_value_history.append(critic_value)

        chosen_action = np.random.choice(num_actions, p=action_probs)
        chosen_action_prob = action_probs[chosen_action]
        logprob = np.log(chosen_action_prob)
        action_probs_history.append(np.log(action_probs[chosen_action]))

        state, reward, done, _ = env.step(chosen_action)
        rewards_history.append(reward)
        episode_reward += reward

        if done:
            break

    # running reward will be used to determine if we should stop training
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

    # TODO: train the actor and critic networks so that they do better in
    # the next iteration

    episode_count += 1
    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

    break # JUST FOR DEMO

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` i

The above code runs the loop in what we call episodes. An episode is just a collection of timesteps. It can go up till `max_steps_per_episode` or it can end early if the agent ends up going to a state where there is nothing left to do.

In cartpole env, an episode ends when the pole falls beyond a certain angle, the cart moves too far from the center, or the maximum number of time steps is reached.

During the episode, we keep track of the actions chosen by the agent, the log probabilities of the chosen action, the critic estimates, and the rewards given back by the env after processing the action.

The agent receives a reward of +1 for each time step that the pole remains upright.

After each episode, we tune the parameters towards a point where the agent actually performs well within the environment. To do that, we need to calculate some loss values for the actor and critic, and then calculate gradients using that so that the loss values can be minimised.

## Training Actor and Critic
Okay, now we need to calculate the loss values for actor and critic.

I will write heavily commented code below to help you understand what's happening. Note that we are using the tape everywhere to keep track of everything needed to calculate the gradients afterwards, as we do these calculations.

In [ ]:
import gym # contains predefined envs that can process the agent's action, and return the rewards and the observations.
import numpy as np
np.random.seed(42)
env = gym.make("CartPole-v0")  # Create the environment
env.seed(42)

# actor and critic will be simple feed forward networks, and they will share the first layer.
num_inputs = 4
num_actions = 2
num_hidden = 128

def init_xavier_weights(shape):
    return np.random.randn(*shape) * np.sqrt(2 / shape[0])

def init_agent_params():
    w_common = init_xavier_weights((num_inputs, num_hidden))
    w_actor = init_xavier_weights((num_hidden, num_actions))
    w_critic = init_xavier_weights((num_hidden, 1))
    return {
        'w_common': w_common,
        'w_actor': w_actor,
        'w_critic': w_critic
    }

agent_params = init_agent_params()

max_steps_per_episode = 10000
gamma = 0.99
eps = np.finfo(np.float32).eps.item() # for softmax
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

# will be used in the calculation of critic loss
def huber_loss_single(y_true, y_pred, delta=1.0):
    # y_true and y_pred are just numbers, not arrays
    error = y_true - y_pred
    abs_error = np.abs(error)
    if abs_error <= delta:
        return 0.5 * error**2
    else:
        return delta * (abs_error - 0.5 * delta)

ctx = {
    'obs': [],
    'common_out': [], # each item in list corresponds to one timestep
    'relu_out': [],
    'actor_logits': [],
    'action_probs': [],
    'chosen_action': [],
    'chosen_action_prob': [],
    'chosen_action_logprob': [],
    'critic_value': [],
    'return': [],
    'ret-val': [],
}

while True:
    observations = env.reset()
    episode_reward = 0
    for timestep in range(1, max_steps_per_episode):
        # forward pass of agent to get action probs and critic value
        obs = observations.reshape(1, -1)
        ctx['obs'].append(obs)
        h = np.dot(obs, agent_params['w_common'])
        ctx['common_out'].append(h)
        h = np.maximum(h, 0)
        ctx['relu_out'].append(h)
        logits = np.dot(h, agent_params['w_actor'])
        ctx['actor_logits'].append(logits)
        action_probs = np.exp(logits) / np.sum(np.exp(logits))
        ctx['action_probs'].append(action_probs)

        critic_value = np.dot(h, agent_params['w_critic'])[0,0]
        ctx['critic_value'].append(critic_value)
        critic_value_history.append(critic_value)

        chosen_action = np.random.choice(num_actions, p=np.squeeze(action_probs))
        ctx['chosen_action'].append(chosen_action)
        chosen_action_prob = action_probs[0, chosen_action]
        ctx['chosen_action_prob'].append(chosen_action_prob)
        logprob = np.log(chosen_action_prob)
        ctx['chosen_action_logprob'].append(logprob)
        action_probs_history.append(logprob)

        observations, reward, done, _ = env.step(chosen_action)
        rewards_history.append(reward)
        episode_reward += reward

        if done:
            break

    # running reward will be used to determine if we should stop training
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

    # train the actor and critic networks so that they do better in
    # the next iteration

    # Calculate expected value from rewards
    # - At each timestep what was the total reward received after that timestep
    # - Rewards in the past are discounted by multiplying them with gamma
    # - These are the labels for our critic - what our critic is expected
    #   to predict accurately.
    returns = []
    discounted_sum = 0
    for r in rewards_history[::-1]:
        discounted_sum = r + gamma * discounted_sum
        returns.insert(0, discounted_sum)

    # Normalize
    returns = np.array(returns)
    returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
    returns = returns.tolist()

    # Calculating loss values to update our network
    history = zip(action_probs_history, critic_value_history, returns)
    actor_losses = []
    critic_losses = []
    for log_prob, value, ret in history:
        # At this point in history, the critic estimated that we would get a
        # total reward = `value` in the future. We took an action with log probability
        # of `log_prob` and ended up receiving a total reward = `ret`.
        # The actor must be updated so that it predicts an action that leads to
        # high rewards (compared to critic's estimate) with high probability.
        ctx['return'].append(ret)
        diff = ret - value
        ctx['ret-val'].append(diff)
        actor_losses.append(-log_prob * diff)  # actor loss

        # The critic must be updated so that it predicts a better estimate of
        # the future rewards.
        l = huber_loss_single(ret, value)
        critic_losses.append(l)

    loss_value = sum(actor_losses) + sum(critic_losses)
    print('loss:', loss_value)
    break # just to test

    episode_count += 1
    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

loss: 27.884028067434656


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Calculating Gradients

Okay now we need to calculate the gradients using chain rule all the way from the loss value till the agent parameters. We have all the values we need stored inside `ctx`. We will go backward one operation at a time and accumulate gradients for each intermiediary tensor, till we reach the agent parameters.

In [ ]:
def calculate_gradients(ctx, agent_params):
    grads = {
        'common_out': [], # each item in list corresponds to one timestep
        'relu_out': [],
        'actor_logits': [],
        'action_probs': [],
        'chosen_action_prob': [],
        'chosen_action_logprob': [],
        'critic_value': [],
        'ret-val': [],
    }

    agent_param_grads = {}
    # initialise with zero gradients using np.zeros_like
    for key in grads:
        for item in ctx[key]:
            grads[key].append(np.zeros_like(item))

    for key in agent_params:
        agent_param_grads[key] = np.zeros_like(agent_params[key])

    # we just go backward one operation at a time
    for step in range(len(ctx['common_out'])):
        # backward for -log_prob * diff
        grads['chosen_action_logprob'][step] += -1 * ctx['ret-val'][step]
        grads['ret-val'][step] += -1 * ctx['chosen_action_logprob'][step]

        # backward for huber loss
        err = ctx['return'][step] - ctx['critic_value'][step]
        if np.abs(err) <= 1.0:
            grads['critic_value'][step] += -1 * err
        else:
            grads['critic_value'][step] += -1 * np.sign(err)

        # backward for diff = ret - value
        grads['critic_value'][step] += -1 * grads['ret-val'][step]

        # backward for np.dot(h, agent_params['w_critic'])
        # h is (1, 128) and w_critic is (128, 1)
        grads['relu_out'][step] += np.dot(grads['critic_value'][step], agent_params['w_critic'].T)
        agent_param_grads['w_critic'] += np.dot(ctx['relu_out'][step].T, grads['critic_value'][step])

        # backward for logprob = np.log(chosen_action_prob)
        grads['chosen_action_prob'][step] += grads['chosen_action_logprob'][step] / ctx['chosen_action_prob'][step]

        # backward for chosing an action
        grads['action_probs'][step][0, ctx['chosen_action'][step]] += grads['chosen_action_prob'][step]

        # backward for softmax
        sum_term = np.sum(grads['action_probs'][step] * ctx['action_probs'][step])
        grads['actor_logits'][step] += ctx['action_probs'][step] * (grads['action_probs'][step] - sum_term)

        # backward for np.dot(relu_out, agent_params['w_actor'])
        # relu_out is (1, 128) and w_actor is (128, 2)
        grads['relu_out'][step] += np.dot(grads['actor_logits'][step], agent_params['w_actor'].T)
        agent_param_grads['w_actor'] += np.dot(ctx['relu_out'][step].T, grads['actor_logits'][step])

        # backward for relu
        grads['common_out'][step] += grads['relu_out'][step] * (ctx['common_out'][step] > 0)

        # backward for np.dot(obs, agent_params['w_common'])
        agent_param_grads['w_common'] += np.dot(ctx['obs'][step].T, grads['common_out'][step])

    return agent_param_grads


grads = calculate_gradients(ctx, agent_params)
for key in grads:
    print(key, grads[key].shape)

w_common (4, 128)
w_actor (128, 2)
w_critic (128, 1)


## Optimisation Step

Now that we have the gradients, we can modify the parameters using a learning rate and the gradients and see if the agent learns over time.

In [ ]:
import gym # contains predefined envs that can process the agent's action, and return the rewards and the observations.
import numpy as np
np.random.seed(42)
env = gym.make("CartPole-v0")  # Create the environment
env.seed(42)

# actor and critic will be simple feed forward networks, and they will share the first layer.
num_inputs = 4
num_actions = 2
num_hidden = 128

def init_xavier_weights(shape):
    return np.random.randn(*shape) * np.sqrt(2 / shape[0])

def init_agent_params():
    w_common = init_xavier_weights((num_inputs, num_hidden))
    w_actor = init_xavier_weights((num_hidden, num_actions))
    w_critic = init_xavier_weights((num_hidden, 1))
    return {
        'w_common': w_common,
        'w_actor': w_actor,
        'w_critic': w_critic
    }

agent_params = init_agent_params()

max_steps_per_episode = 10000
gamma = 0.99
eps = np.finfo(np.float32).eps.item() # for softmax
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

# will be used in the calculation of critic loss
def huber_loss_single(y_true, y_pred, delta=1.0):
    # y_true and y_pred are just numbers, not arrays
    error = y_true - y_pred
    abs_error = np.abs(error)
    if abs_error <= delta:
        return 0.5 * error**2
    else:
        return delta * (abs_error - 0.5 * delta)


while True:
    observations = env.reset()
    episode_reward = 0
    ctx = {
        'obs': [],
        'common_out': [], # each item in list corresponds to one timestep
        'relu_out': [],
        'actor_logits': [],
        'action_probs': [],
        'chosen_action': [],
        'chosen_action_prob': [],
        'chosen_action_logprob': [],
        'critic_value': [],
        'return': [],
        'ret-val': [],
    }
    for timestep in range(1, max_steps_per_episode):
        # forward pass of agent to get action probs and critic value
        obs = observations.reshape(1, -1)
        ctx['obs'].append(obs)
        h = np.dot(obs, agent_params['w_common'])
        ctx['common_out'].append(h)
        h = np.maximum(h, 0)
        ctx['relu_out'].append(h)
        logits = np.dot(h, agent_params['w_actor'])
        ctx['actor_logits'].append(logits)
        action_probs = np.exp(logits) / np.sum(np.exp(logits))
        ctx['action_probs'].append(action_probs)

        critic_value = np.dot(h, agent_params['w_critic'])[0,0]
        ctx['critic_value'].append(critic_value)
        critic_value_history.append(critic_value)

        chosen_action = np.random.choice(num_actions, p=np.squeeze(action_probs))
        ctx['chosen_action'].append(chosen_action)
        chosen_action_prob = action_probs[0, chosen_action]
        ctx['chosen_action_prob'].append(chosen_action_prob)
        logprob = np.log(chosen_action_prob)
        ctx['chosen_action_logprob'].append(logprob)
        action_probs_history.append(logprob)

        observations, reward, done, _ = env.step(chosen_action)
        rewards_history.append(reward)
        episode_reward += reward

        if done:
            break

    # running reward will be used to determine if we should stop training
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

    # train the actor and critic networks so that they do better in
    # the next iteration

    # Calculate expected value from rewards
    # - At each timestep what was the total reward received after that timestep
    # - Rewards in the past are discounted by multiplying them with gamma
    # - These are the labels for our critic - what our critic is expected
    #   to predict accurately.
    returns = []
    discounted_sum = 0
    for r in rewards_history[::-1]:
        discounted_sum = r + gamma * discounted_sum
        returns.insert(0, discounted_sum)

    # Normalize
    returns = np.array(returns)
    returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
    returns = returns.tolist()

    # Calculating loss values to update our network
    history = zip(action_probs_history, critic_value_history, returns)
    actor_losses = []
    critic_losses = []
    for log_prob, value, ret in history:
        # At this point in history, the critic estimated that we would get a
        # total reward = `value` in the future. We took an action with log probability
        # of `log_prob` and ended up receiving a total reward = `ret`.
        # The actor must be updated so that it predicts an action that leads to
        # high rewards (compared to critic's estimate) with high probability.
        ctx['return'].append(ret)
        diff = ret - value
        ctx['ret-val'].append(diff)
        actor_losses.append(-log_prob * diff)  # actor loss

        # The critic must be updated so that it predicts a better estimate of
        # the future rewards.
        l = huber_loss_single(ret, value)
        critic_losses.append(l)

    loss_value = sum(actor_losses) + sum(critic_losses)

    grads = calculate_gradients(ctx, agent_params)
    # update params
    lr = 0.005
    for key in agent_params:
        agent_params[key] -= lr * grads[key]

    action_probs_history.clear()
    critic_value_history.clear()
    rewards_history.clear()

    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 13.63 at episode 10
running reward: 17.57 at episode 20
running reward: 16.45 at episode 30
running reward: 14.06 at episode 40
running reward: 13.27 at episode 50
running reward: 12.35 at episode 60
running reward: 12.22 at episode 70
running reward: 11.97 at episode 80
running reward: 12.28 at episode 90
running reward: 11.88 at episode 100
running reward: 12.29 at episode 110
running reward: 11.75 at episode 120
running reward: 12.35 at episode 130
running reward: 12.85 at episode 140
running reward: 12.50 at episode 150
running reward: 12.96 at episode 160
running reward: 12.30 at episode 170
running reward: 11.65 at episode 180
running reward: 13.00 at episode 190
running reward: 14.14 at episode 200
running reward: 13.92 at episode 210
running reward: 12.59 at episode 220
running reward: 12.70 at episode 230
running reward: 12.54 at episode 240
running reward: 13.69 at episode 250
running reward: 13.64 at episode 260
running reward: 13.35 at episode 270
running re

## Intuitions

### How does the actor learn?

The overall purpose of actor network is easy to understand - it needs to pick the best possible actions so that the agent gets high rewards over time.

But how does the actor loss value help with that? The loss value is calculated as:

```
actor_loss = -log_prob * (return - critic_value)
```

Let's focus on the `return - critic_value` first. The reason we have the critic's estimate is to figure out how good or bad the current state of the environment is. Not every situation inside an environment is one where there is an action that will result in high rewards. Maybe the agent is in a really bad situation, where they will get negative rewards no matter what action they choose. A critic will judge the situation and output an estimate of the rewards they can get in the future. And the agent should be tuned based on the difference of the actual rewards they ended up getting and the critic's estimate.

The `logprob` represents the probability that the actor assigned to chosen action. Now, if the action led to good rewards, the difference between actual rewards and critic's estimate will be high. If the probability predicted for this action was high, `logprob` will be close to 0. Therefore, the product will be close to 0. If the probability was low, the loss will be high.

If the action led to bad rewards, the difference between the actual rewards and the critic's estimate will be negative. If the probability for this bad action was low, the loss value will be high.

So tuning the parameters in a way that brings this loss down will help the agent learn the correct behavior.

### How does the critic learn?

Critic is more straightforward. At the end of the episode, we know the total rewards the agent ended up getting at each timestep. So, we can simply use a loss function based on the difference between the critic's predicted estimate and the actual rewards. We use huber loss here.

The rewards in the later timesteps are discounted to reflect the fact that the rewards which immediately follow the current state are more important than the rewards in the far future.

Let me spell that out in a better way:

At timestep `t`: The sum of rewards from this point onwards will be `r(t) + r(t+1) + ...`. We discount the future rewards by modifying this sum to:
```
r(t) + gamma * r(t+1) + gamma * gamma * r(t+2) + ...
```
where `gamma` is something like 0.99



## Thank you

Hit me up on [X](https://x.com/_apoorvnandan) if you any questions or feedback!

[Buy me a coffee](https://buymeacoffee.com/apoorvn) if you would like to support my work!